# Playing Differentiated Demand Environemnt

In [1]:
#Imports

from marketsai.markets.diff_demand import DiffDemand

#import ray

from ray import tune, shutdown, init
from ray.tune.registry import register_env
from ray.rllib.agents.a3c.a2c import A2CTrainer
from ray.rllib.agents.dqn.dqn import DQNTrainer
from ray.tune.integration.mlflow import MLflowLoggerCallback
from ray.rllib.utils.schedules.exponential_schedule import ExponentialSchedule

import random
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import logging

/Users/matiascovarrubias/Library/Caches/pypoetry/virtualenvs/marketsai-reVLCGV_-py3.8/lib/python3.8/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
Instructions for updating:
non-resource variables are not supported in the long term
{'agent_0': array([1.6, 1.6]), 'agent_1': array([1.6, 1.6])} {'agent_0': 0.27249236968976237, 'agent_1': 0.27249236968976237} {'__all__': False} {'agent_0': 1.6, 'agent_1': 1.6}


In [2]:
# STEP 0: Inititialize ray
NUM_CPUS = 12
shutdown()
init(num_cpus=NUM_CPUS, 
    logging_level=logging.ERROR,
)

{'node_ip_address': '192.168.1.202',
 'raylet_ip_address': '192.168.1.202',
 'redis_address': '192.168.1.202:18536',
 'object_store_address': '/tmp/ray/session_2021-04-11_12-42-56_244925_76877/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-04-11_12-42-56_244925_76877/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-04-11_12-42-56_244925_76877',
 'metrics_export_port': 64203,
 'node_id': 'ab4292b0ee968b2d660073a3fe4930d67935de526069de415bb0aaa6'}

In [3]:
# STEP 1: register environment
register_env("diffdemand", DiffDemand)
env = DiffDemand()
policy_ids = [f"policy_{i}" for i in range(env.n_agents)]

In [4]:
# STEP 2: Experiment configuration

MAX_STEPS = 20 * 1000
PRICE_BAND_WIDE = 0.1
LOWER_PRICE = 1.47 - PRICE_BAND_WIDE
HIGHER_PRICE = 1.93 + PRICE_BAND_WIDE
DEC_RATE = math.e ** (-4 * 10 ** (-6))
DEC_RATE_HIGH = math.e ** (-4 * 10 ** (-6) * 4)

env_config = {
    "mkt_config": {
    "lower_price": [LOWER_PRICE for i in range(env.n_agents)],
    "higher_price": [HIGHER_PRICE for i in range(env.n_agents)],
    "parameteres": {
                "cost": [1 for i in range(env.n_agents)],
                "values": [2 for i in range(env.n_agents)],
                "ext_demand": 0,
                "substitution": 0.25,
    },
    "space_type": "MultiDiscrete",
    "gridpoints": 16,
    }
}

exploration_config = {
    "type": "EpsilonGreedy",
    "epsilon_schedule": ExponentialSchedule(
        schedule_timesteps=1,
        framework=None,
        initial_p=1,
        decay_rate=DEC_RATE,
    ),
}

training_config = {
    "gamma": 0.95,
    "lr": 0.15,
    "env": "diffdemand",
    "exploration_config": exploration_config,
    "env_config": env_config,
    "horizon": 100,
    "soft_horizon": True,
    "no_done_at_end": True,
    "multiagent": {
        "policies": {
            policy_ids[i]: (
                None,
                env.observation_space["agent_{}".format(i)],
                env.action_space["agent_{}".format(i)],
                {},
            )
            for i in range(env.n_agents)
        },
        "policy_mapping_fn": (lambda agent_id: policy_ids[int(agent_id.split("_")[1])]),
    },
    "framework": "torch",
    "num_workers": NUM_CPUS - 1,
    "num_gpus": 0,
}

stop = {"info/num_steps_trained": MAX_STEPS}

In [5]:
#Step 3: Experiments

exp_name = "DQN_test_April9"
results = tune.run(
    "DQN",
    name=exp_name,
    config=training_config,
    checkpoint_freq=250,
    checkpoint_at_end=True,
    stop=stop,
    metric="episode_reward_mean",
    mode="max",
    callbacks=[MLflowLoggerCallback(experiment_name=exp_name, save_artifact=True)],
)

best_checkpoint = results.best_checkpoint
print("Best checkpont:", best_checkpoint)

Trial name,status,loc
DQN_diffdemand_f918e_00000,RUNNING,


rrubias/Library/Caches/pypoetry/virtualenvs/marketsai-reVLCGV_-py3.8/lib/python3.8/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
(pid=78908)   warnings.warn(msg)
(pid=78903) /Users/matiascovarrubias/Library/Caches/pypoetry/virtualenvs/marketsai-reVLCGV_-py3.8/lib/python3.8/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
(pid=78903)   warnings.warn(msg)
(pid=78906) WARNING:tensorflow:From /Users/matiascovarrubias/Library/Caches/pypoetry/virtualenvs/marketsai-reVLCGV_-py3.8/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=78906) Instructions for upda

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
DQN_diffdemand_f918e_00000,RUNNING,192.168.1.202:78912,1,0.661942,1012,nan,nan,nan,nan


Result for DQN_diffdemand_f918e_00000:
  custom_metrics: {}
  date: 2021-04-11_12-43-11
  done: false
  episode_len_mean: 100.0
  episode_reward_max: 55.75359489802493
  episode_reward_mean: 54.01029982842044
  episode_reward_min: 51.842024196182635
  episodes_this_iter: 11
  episodes_total: 55
  experiment_id: ee9e90f38ff94c029b1754ffc1ecea6a
  hostname: Matiass-MBP.fios-router.home
  info:
    last_target_update_ts: 5764
    learner:
      policy_0:
        allreduce_latency: 0.0
        cur_lr: 0.15
        grad_gnorm: 6.867670059204102
        max_q: 20.781890869140625
        mean_q: 20.386672973632812
        mean_td_error: -0.8119003176689148
        min_q: 20.08861541748047
      policy_1:
        allreduce_latency: 0.0
        cur_lr: 0.15
        grad_gnorm: 10.586507797241211
        max_q: 29.594013214111328
        mean_q: 29.096073150634766
        mean_td_error: -1.4710311889648438
        min_q: 28.4183292388916
    num_steps_sampled: 6072
    num_steps_trained: 3712
  

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
DQN_diffdemand_f918e_00000,RUNNING,192.168.1.202:78912,6,6.18618,6072,54.0103,55.7536,51.842,100


...

Result for DQN_diffdemand_f918e_00000:
  custom_metrics: {}
  date: 2021-04-11_12-43-39
  done: false
  episode_len_mean: 100.0
  episode_reward_max: 56.05073768323267
  episode_reward_mean: 53.78382867242093
  episode_reward_min: 51.003203070556395
  episodes_this_iter: 11
  episodes_total: 264
  experiment_id: ee9e90f38ff94c029b1754ffc1ecea6a
  hostname: Matiass-MBP.fios-router.home
  info:
    last_target_update_ts: 26884
    learner:
      policy_0:
        allreduce_latency: 0.0
        cur_lr: 0.15
        grad_gnorm: 0.17977513372898102
        max_q: 8.541720390319824
        mean_q: 8.428948402404785
        mean_td_error: -0.06360691785812378
        min_q: 8.32476806640625
      policy_1:
        allreduce_latency: 0.0
        cur_lr: 0.15
        grad_gnorm: 0.8584436178207397
        max_q: 12.674427032470703
        mean_q: 12.554797172546387
        mean_td_error: 0.14714717864990234
        min_q: 12.41058349609375
    num_steps_sampled: 27324
    num_steps_trained: 191

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
DQN_diffdemand_f918e_00000,RUNNING,192.168.1.202:78912,27,33.2925,27324,53.7838,56.0507,51.0032,100


Result for DQN_diffdemand_f918e_00000:
  custom_metrics: {}
  date: 2021-04-11_12-43-43
  done: true
  episode_len_mean: 100.0
  episode_reward_max: 56.05073768323267
  episode_reward_mean: 53.68502238109662
  episode_reward_min: 51.003203070556395
  episodes_this_iter: 11
  episodes_total: 286
  experiment_id: ee9e90f38ff94c029b1754ffc1ecea6a
  hostname: Matiass-MBP.fios-router.home
  info:
    last_target_update_ts: 28996
    learner:
      policy_0:
        allreduce_latency: 0.0
        cur_lr: 0.15
        grad_gnorm: 0.3377199172973633
        max_q: 8.102686882019043
        mean_q: 8.039983749389648
        mean_td_error: 0.12461180984973907
        min_q: 7.962584018707275
      policy_1:
        allreduce_latency: 0.0
        cur_lr: 0.15
        grad_gnorm: 0.2324407547712326
        max_q: 10.982880592346191
        mean_q: 10.808721542358398
        mean_td_error: -0.041232407093048096
        min_q: 10.645918846130371
    num_steps_sampled: 29348
    num_steps_trained: 20

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
DQN_diffdemand_f918e_00000,TERMINATED,,29,36.5148,29348,53.685,56.0507,51.0032,100


Best checkpont: /Users/matiascovarrubias/ray_results/DQN_test_April9/DQN_diffdemand_f918e_00000_0_2021-04-11_12-42-58/checkpoint_29/checkpoint-29


## Continuous Space


In [6]:
env_config["mkt_config"]["space_type"] = "Continuous"
env=DiffDemand(env_config)
training_config["env_config"] = env_config
training_config["multiagent"]["policies"] =  {
            policy_ids[i]: (None, env.observation_space[f"agent_{i}"], env.action_space[f"agent_{i}"],{},) for i in range(env.n_agents)
}
#print(env_config)
print(training_config)
print(env.action_space)


{'gamma': 0.95, 'lr': 0.15, 'env': 'diffdemand', 'exploration_config': {'type': 'EpsilonGreedy', 'epsilon_schedule': <ray.rllib.utils.schedules.exponential_schedule.ExponentialSchedule object at 0x192b5d7c0>}, 'env_config': {'mkt_config': {'lower_price': [1.3699999999999999, 1.3699999999999999], 'higher_price': [2.03, 2.03], 'parameteres': {'cost': [1, 1], 'values': [2, 2], 'ext_demand': 0, 'substitution': 0.25}, 'space_type': 'Continuous', 'gridpoints': 16}}, 'horizon': 100, 'soft_horizon': True, 'no_done_at_end': True, 'multiagent': {'policies': {'policy_0': (None, Box(1.3699999999999999, 2.03, (2,), float64), Box(1.3699999999999999, 2.03, (1,), float64), {}), 'policy_1': (None, Box(1.3699999999999999, 2.03, (2,), float64), Box(1.3699999999999999, 2.03, (1,), float64), {})}, 'policy_mapping_fn': <function <lambda> at 0x192b3e1f0>}, 'framework': 'torch', 'num_workers': 11, 'num_gpus': 0}
{'agent_0': Box(1.3699999999999999, 2.03, (1,), float64), 'agent_1': Box(1.3699999999999999, 2.03,

In [7]:
exp_name = "DDPG_cont_test_April9"
results = tune.run(
    "DDPG",
    name=exp_name,
    config=training_config,
    checkpoint_freq=250,
    checkpoint_at_end=True,
    stop=stop,
    metric="episode_reward_mean",
    mode="max",
    callbacks=[MLflowLoggerCallback(experiment_name=exp_name, save_artifact=True)],
)

best_checkpoint = results.best_checkpoint
print("Best checkpont:", best_checkpoint)

Trial name,status,loc
DDPG_diffdemand_13c23_00000,RUNNING,


ble before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:143.)
(pid=78987)   torch.from_numpy(self.action_space.low).float())
(pid=78987) 2021-04-11 12:43:52,760	WARNING deprecation.py:33 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
(pid=78981) {'agent_0': array([1.6, 1.6]), 'agent_1': array([1.6, 1.6])} {'agent_0': 0.27249236968976237, 'agent_1': 0.27249236968976237} {'__all__': False} {'agent_0': 1.6, 'agent_1': 1.6}
(pid=78982) {'agent_0': array([1.6, 1.6]), 'agent_1': array([1.6, 1.6])} {'agent_0': 0.27249236968976237, 'agent_1': 0.27249236968976237} {'__all__': False} {'agent_0': 1.6, 'agent_1': 1.6}
(pid=78983) {'agent_0': array([1.6, 1.6]), 'agent_1': array([1.6, 1.6])} {'agent_0': 0.27249236968976237, 'agent_1': 0.27249236968976237} {'__all__': False} {'agent_0': 1.6, 'agent_1':

Trial name,status,loc
DDPG_diffdemand_13c23_00000,ERROR,
Trial name,# failures,error file
DDPG_diffdemand_13c23_00000,1,/Users/matiascovarrubias/ray_results/DDPG_cont_test_April9/DDPG_diffdemand_13c23_00000_0_2021-04-11_12-43-43/error.txt


Trial name,status,loc
DDPG_diffdemand_13c23_00000,ERROR,
Trial name,# failures,error file
DDPG_diffdemand_13c23_00000,1,/Users/matiascovarrubias/ray_results/DDPG_cont_test_April9/DDPG_diffdemand_13c23_00000_0_2021-04-11_12-43-43/error.txt


TuneError: ('Trials did not complete', [DDPG_diffdemand_13c23_00000])